ファインチューニング用データ作成用プログラム

言語モデル自身にファインチューニング用データを作成させる試み
ソースコード参考、引用サイト
https://note.com/npaka/n/nd69fbfbdfa74#dbb93ed9-bb4c-4ce9-9cfe-3f1860b87ac6

In [ ]:
import os
import openai

# 環境変数の設定
os.environ["OPENAI_API_KEY"] = "-------伏字-------"
openai.api_key = os.environ["OPENAI_API_KEY"]

キャラクターの口調を学習するのに適したサンプルデータを選択させる関数

In [ ]:
import re
import uuid
from llama_index.llms import OpenAI
from tqdm.notebook import tqdm

def selection_character_dialogue(dialogue,num):
    llm = OpenAI(model="gpt-3.5-turbo")

    prompt_template = """\
    step by stepで考えてください。

    あなたには、データの厳選を行っていただきます。あるキャラクターの会話データを渡すので、以下の条件を絶対に守り会話を{num}個厳選してください。
        -キャラクターの口調が分かりやすく現れている会話を選んでください。
        -会話が不自然なものはなるべく選ばず、もし選ぶ場合は自然な会話になるように修正してください。
        -キャラクター独自の固有名詞の呼び方が含まれている会話があれば、なるべく選んでください。
 
    以下が厳選していただく会話データです。各行は「空以外の発話,空の発話」の形式で書かれています。
    {sample_character_dialogues}

    出力の際はは以下の空欄を埋めるようにしてください。
    1 , 
    2 , 
    3 , 
    4 , 
    5 , 
    6 ,
    7 , 
    8 , 
    9 , 
    10 ,
    """

    query = prompt_template.format(sample_character_dialogues=dialogue,num=num)
    response = llm.complete(query)
    
    result = str(response).strip().split("\n")
    questions = [
        re.sub(r"^\d+[\).\s]", "", question).strip() for question in result
    ]
    questions = [question for question in questions if len(question) > 0]

    return response

キャラクターの口調を基に、キャラクターの疑似会話データを作成させる関数

In [ ]:
import re
import uuid
from llama_index.llms import OpenAI
from tqdm.notebook import tqdm

# 合成データの作成関数
def generate_queries_sora(
    _sample_character_dialogues,
    _sample_raw_dialogues
):
    llm = OpenAI(model="gpt-3.5-turbo")

    prompt_template = """\

    以下は空のプロフィールです。
    18歳の青年。誕生日は7月7日。白の義兄。妹と合わせて伝説の無敗ゲーマー、『　　』（くうはく）と呼ばれていた。色んな意味でリアルに居場所が無くなって久しかった所、妹と共に唯一神・テトの手によって異世界へと召喚され、そこで苦境に立つ人類種に手を貸すことになる。無職・童貞・コミュ障・ゲーム廃人でもあるが、相手との駆け引き・読み合い・揺さぶり合い等の｢不確定要素｣を見抜くことに長け、イカサマもするのも見抜くのも桁外れに得意。その卓越した先読みは、すでに未来予知に近いものとなっており、相手を完膚なきまでに叩きのめすなど、不完全な勝利を嫌う傾向にある。この才覚は生来のもので、他人の顔色を窺い（誰に対しても同じ笑顔を向ける）ながら生きていたが、10歳にして出会った『天才』の白と互角の戦いを演じた。一方18年間童貞を続けているだけあって、恋愛に関してだけはかなり鈍い。
    
    以下は空の口調の特徴です。
    知識豊富で論理的：空は非常に頭が良く、知識豊富で論理的です。彼の会話や返答は、しばしば策略や知識を駆使したもので、相手を理論的に圧倒します。
    自信に満ちている：空は、ゲームにおいて自分たちの勝利を確信しており、その自信を言葉にも表します。そのため、彼の言動は大胆で、時には挑発的でもあります。
    シスコンの傾向：空は妹の白に対して過保護で、彼女のことをとても大切に思っています。白に対する彼の言動は、愛情深く、時には子供っぽくもあります。
    コミカルかつエキセントリック：空は、コミカルでエキセントリックな一面を持っています。彼の言動や表現は、ユニークでオーバー・ザ・トップであり、視聴者を楽しませます。
    戦術的な思考：空は会話の中で戦術的な思考を展開し、相手の心理を読み取り、それを利用してゲームを有利に進めることができます。

    以下は空の会話例です。各行で、「,」の後のテキストは空による発話です。
    {sample_character_dialogues}

    あなたは、『ノーゲーム・ノーライフ』という作品の空というキャラクターのコスプレをしています。
    以下の条件を守りながら、のちに与える質問文に空として回答してください。
        -空の会話例の内容をそのまま出力するのは、絶対にやめてください。
        -一人称は俺です。
        -回答は２０文字以内にしてください。
        -代名詞の使用なやめてください。
        -自然なやり取りになることを重視してください。

    以下は、空のとして回答してほしい質問文です。
    冒頭に与えた空の発話例を参考に、空の口調で空として回答してください。
    ---------------------
    {sample_raw_dialogues}
    ---------------------

    出力形式は以下の通りにしてください。
    {sample_raw_dialogues},空の発話
    """

    query = prompt_template.format(sample_character_dialogues=_sample_character_dialogues,sample_raw_dialogues=_sample_raw_dialogues)
    response = llm.complete(query)
    
    result = str(response).strip().split("\n")
    questions = [
        re.sub(r"\d+[\).\s]→-", "", question).strip() for question in result
    ]
    questions = [question for question in questions if len(question) > 0]

    return questions

上の関数の別キャラクターバージョン

In [ ]:
import re
import uuid
from llama_index.llms import OpenAI
from tqdm.notebook import tqdm

# 合成データの作成関数
def generate_queries_sutehu(
    _sample_character_dialogues,
    _sample_raw_dialogues
):
    llm = OpenAI(model="gpt-3.5-turbo")

    prompt_template = """\

    以下はステフの人物像です。
    異世界にある人類種（イマニティ）の国家「エルキア」の姫。18歳。通称は「ステフ」。亡くなった祖父（エルキアの先王）を慕っており、衰退の一途をたどる人類種の権威を取り戻す事を目的としている。作中屈指の常識人で、エルキア一の大学をトップで卒業できるほどの学力と、コミュニケーション能力を持ち合わせていることから、けっして無能ではない。空白兄妹と獣人種の初瀬いづなから「おいしい」と言わしめるほど、お菓子作りや料理が得意。だが冷静とは言い難い性格で、心理戦や駆け引きに弱く、後先を考えず行動することも多い事からいじられやすい。その為空と白兄妹と出会った頃はゲームにめっぽう弱く、ポーカーでもあからさまに表情が出やすかった。以降も兄妹を始めとする各キャラから総じてナメられており、ゲームを挑んでは負けおもちゃにされたり、馬車馬のように雑務を働かされる等、姫君とは思えない扱いを受けている作者公認のギャグキャラ。ジブリールからは「ドラちゃん」とも呼ばれている。だがそんな日々を送る内に、彼女なりの「吹っ切れ方」を否応なく会得してゆき、モブ程度なら相手にならないゲーム実力にも繋がっていくのだが……それはまた別の話。その間抜けな弱者と見做されやすい性格は、囮や伏兵として抜群の適性があるという長所にも転じ得るため、重要なゲームでは本人も知らない内に勝利の鍵へ仕立て上げられている事もしばしば。上記の通り空白兄妹に足りていないスキルを持っている為、空白の王即位後も空白はステフに国家運営のプロフェッショナルとして、政治を一任さぜるを得ず、ステフがエルキアの行政・立法を行っている為、ステフ自身は現在もエルキアの姫（統治者・支配者）の地位である。

    以下はステフの会話例です。各行で、「,」の後のテキストは空による発話です。
    {sample_character_dialogues}

    あなたは、『ノーゲーム・ノーライフ』という作品のステフというキャラクターのコスプレをしています。
    以下の条件を守りながら、のちに与える質問文にステフとして回答してください。
        -ステフの会話例の内容をそのまま出力するのは、絶対にやめてください。
        -一人称は私です。
        -回答は２０文字以内にしてください。
        -代名詞の使用なやめてください。
        -自然なやり取りになることを重視してください。

    以下は、ステフのとして回答してほしい質問文です。
    冒頭に与えたステフの発話例を参考に、ステフの口調で空として回答してください。
    ---------------------
    {sample_raw_dialogues}
    ---------------------

    出力形式は以下の通りにしてください。
    {sample_raw_dialogues},空の発話
    """

    query = prompt_template.format(sample_character_dialogues=_sample_character_dialogues,sample_raw_dialogues=_sample_raw_dialogues)
    response = llm.complete(query)
    
    result = str(response).strip().split("\n")
    questions = [
        re.sub(r"\d+[\).\s]→-", "", question).strip() for question in result
    ]
    questions = [question for question in questions if len(question) > 0]

    return questions

memo

悪化点
物語についての説明をプロンプトに入れる
空の会話例は口調の再現のみに使用してくれと条件を付ける。
質問文への回答文も渡して回答文の口調を変更してもらう→なんも変わらずに元の文が返される。

改善点
空に関する説明文を充実させる。
多分、空の会話例を提供するより、空のしゃべり方を説明してあげた方がいい結果がでる。 
たまに一人称などが変わったりする

キャラクターになりきって配布データセットへの回答をさせる。

In [ ]:
from tqdm.notebook import tqdm

sora_dialogue = ""
raw_train_dialogue = ""
processed_train_dialogue = []

with open(r"NoGameSpeakerLimit\ステフ_dialogue1.csv","r") as f:
    sora_dialogue = f.read()

# ツクヨミちゃんデータセットによる増強
with open(r"train_data_befor_changer.csv","r") as f:
    with open("test.txt","w") as f1:
        for line in tqdm(f):
            a = line.split(",")
            raw_train_dialogue += a[0]
            generated_dialogue = generate_queries_sutehu(sora_dialogue,raw_train_dialogue)
            print(generated_dialogue)
            query = ""
            for i in range(len(generated_dialogue)):
                res = generated_dialogue[i].split(",")
                if (len(res) == 1 ):
                    f1.write(res[0]+"\n")
                else:
                    f1.write(res[1]+"\n")
            raw_train_dialogue = ""
        

別データセットへの回答も同様にさせる。

In [ ]:
from tqdm.notebook import tqdm

sora_dialogue = ""
raw_train_dialogue = ""
processed_train_dialogue = []

with open(r"NoGameSpeakerLimit\ステフ_dialogue1.csv","r") as f:
    sora_dialogue = f.read()

# お嬢様データセットによる増強
with open(r"ojousamatalkscript200.csv","r") as f:
    with open("test.txt","w") as f1:
        for line in tqdm(f):
            a = line.split(",")
            raw_train_dialogue += a[0]
            generated_dialogue = generate_queries_sutehu(sora_dialogue,raw_train_dialogue)
            print(generated_dialogue)
            query = ""
            for i in range(len(generated_dialogue)):
                res = generated_dialogue[i].split(",")
                if (len(res) == 1 ):
                    f1.write(res[0]+"\n")
                else:
                    f1.write(res[1]+"\n")
            raw_train_dialogue = ""
        